## Neural Network Model 

This neaural network model will predict weither the coffee is rosted good or bad using two perimeters `temperature` and `duration`. we will use tensor flow to achive the objective 

In [18]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

We willl use tensor flow to create a neural network model in tensorflow for coffee rosting dataset. First we will create a dataset using the function below

In [19]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [20]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

(200, 2) (200, 1)


In [21]:
print(f"Maximum Temperature, Minimum temperature :, {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Maximum Duration, Minimum Duration :, {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")

Maximum Temperature, Minimum temperature :, 284.99, 151.32
Maximum Duration, Minimum Duration :, 15.45, 11.51


#### Scaling the data
We will scale the data of features before using it on the model. We have used normalization method to scale the data 

In [22]:
norm_l=tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)
Xn=norm_l(X)

In [11]:
print(f"Maximum Temperature after normalization, Minimum temperature after Normalization :, {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Maximum Duration after Normalization, Minimum Duration after Normalization :, {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

Maximum Temperature after normalization, Minimum temperature after Normalization :, 1.66, -1.69
Maximum Duration after Normalization, Minimum Duration after Normalization :, 1.79, -1.70


`We will increase the size of the dataset to 200,000 rows 

In [23]:
Xt = np.tile(Xn,(1000,1))
Yt= np.tile(Y,(1000,1))   
print(Xt.shape, Yt.shape)   

(200000, 2) (200000, 1)


 In the above step we have increased the dataset from 2000 to 200,000 number of samples. Now we will instantiate the model

In [24]:
tf.random.set_seed(1234) #applied to achive consistent results
model=Sequential(
    [
    tf.keras.Input(shape=(2,)),
    Dense(3, activation='sigmoid', name='layer1'),
    Dense(1, activation='sigmoid', name='layer2')              
    ]
)


In [25]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [26]:
W1, b1 = model.get_layer('layer1').get_weights()
W2, b2 = model.get_layer('layer2').get_weights()

In [28]:
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[ 0.55199647 -0.71500456  0.9824865 ]
 [-0.58328986 -0.5577629  -0.14954174]] 
b1: [0. 0. 0.]
W2:
 [[1.1021022 ]
 [0.24180949]
 [0.7425983 ]] 
b2: [0.]


Now we will compile the Model and fit it

In [29]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10
6250/6250 [==============================] - 6s 914us/step - loss: 0.2200
Epoch 2/10
6250/6250 [==============================] - 6s 916us/step - loss: 0.1450
Epoch 3/10
6250/6250 [==============================] - 6s 898us/step - loss: 0.1386
Epoch 4/10
6250/6250 [==============================] - 6s 905us/step - loss: 0.1360
Epoch 5/10
6250/6250 [==============================] - 6s 927us/step - loss: 0.1342
Epoch 6/10
6250/6250 [==============================] - 6s 928us/step - loss: 0.1331
Epoch 7/10
6250/6250 [==============================] - 6s 926us/step - loss: 0.1324
Epoch 8/10
6250/6250 [==============================] - 6s 975us/step - loss: 0.1317
Epoch 9/10
6250/6250 [==============================] - 6s 936us/step - loss: 0.1313
Epoch 10/10
6250/6250 [==============================] - 6s 955us/step - loss: 0.1309


### Updated weights for the Model

In [30]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[  5.8449574  10.809995    8.806063 ]
 [-15.924869    7.825997    2.5995893]] 
b1: [18.545805  -1.4267157 12.128967 ]
W2:
 [[  7.0576506]
 [-22.796059 ]
 [  9.7351885]] 
b2: [-14.715465]


You can see that the values are different from what you printed before calling model.fit(). With these, the model should be able to discern what is a good or bad coffee roast.
For the purpose of the next discussion, instead of using the weights you got right away, you will first set some weights we saved from a previous training run. This is so that this notebook remains robust to changes in Tensorflow over time. Different training runs can produce somewhat different results and the following discussion applies when the model has the weights you will load below.
Feel free to re-run the notebook later with the cell below commented out to see if there is any difference. If you got a low loss after the training above (e.g. 0.002), then you will most likely get the same results.

### Predictions for the Model

We will make predicions on the test data which has to go through a simlr process of normalization before testing it with the model. here we are predictiing on the test data as given below

In [31]:
X_test = np.array([
    [200,13.9],  # positive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 202ms/step
predictions = 
 [[7.6872814e-01]
 [8.6474935e-13]]


In [32]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


The model predicted the first row as positive and the second row is negative